# Testing your model works before submisson

In order to make sure that your model will work when tested, you should run this notebook. Take the following steps:
1. Have your model.dat file placed in a directory of your choosing. 
2. Paste the code of your PolyNet class in the second to last cell of this notebook. ** Make sure your PolyNet class code matches the code used to generate model.dat!**
3. In the last cell, make sure to load your model from the right directory
4. Run the this whole notebook. Then verify that target polynomials and network's polynomials are drawn in files located at "./test/draw" directory



In [1]:
import numpy as np
import PIL.Image as Image
from PIL import ImageDraw

import torch.nn as nn

import torch
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
import os
import glob
import datetime

# Device configuration
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [2]:
class PolyDataset(Dataset):
    
    def __init__(self, dataset_dir):
        """
        Initializing dataset by generating a dicitonary of labels, where an image file name is the key 
        and its labels are the contents of that entry in the dictionary. Images are not loaded. This way it
        is possible to iterate over arbitrarily large datasets (limited by labels dicitonary fitting 
        in memory, which is not a problem in practice)
        
        Args:
            dataset_dir : path to directory with images and labels. In this directory we expect to find
                          a directory called "images" containing the input images, and a file called 
                          "labels.txt" containing desired labels (coefficients)
        """
        
        self.dataset_dir = dataset_dir
        self.labels_dict = self.gen_labels_dict()
        self.images_keys = list(self.labels_dict)  # getting the keys of the dictionary as list
        self.images_keys.sort()                    # sorting so as to have in alphabetical order 

    def __len__(self):
        return len(self.labels_dict)

    def __getitem__(self, index):    
        """
        This funtion makes it possible to iterate over the PolyDataset
        Args:
            index: running index of images
            
        Returns:
            sample: a dicitionary with three entries:
                    1. 'image'  contains the image
                    2. 'labels' contains labels (coeffs) corresponding to image
                    3. 'fname'  contains name of file (image_key) - may be useful for debugging
        """
        image_key = self.images_keys[index]     # recall - key is the file name of the corresponding image
        image = np.array(Image.open(image_key)) # image has shape: (128, 128, 3)
        image = image/255.0                     # simple normalization - just to maintain small numbers
        image = np.transpose(image, (2, 0, 1))  # network needs RGB channels to be first index
        labels = self.labels_dict[image_key]
        sample = {'image': image, 'labels': labels, 'fname':image_key}
        
        return sample
    
    
    def gen_labels_dict(self):
        """
        This fucntion generates a dictionary of labels
        
        Returns:
            labels_dict: the key is image file name and an array of labels is the corresponding contents 
        """
        
        labels_fname = self.dataset_dir + "/labels.txt"
        labels_dict = {}
        with open(labels_fname, "r") as inp:
            for line in inp:
                line = line.split('\n')[0]                                      # remove '\n' from end of line 
                line = line.split(',')
                key  = self.dataset_dir + '/images/' + line[0].strip() + ".png" # image file name is the key
                del line[0]
                
                list_from_line = [float(item) for item in line]
                labels_dict[key] = np.asarray(list_from_line, dtype=np.float32)
                        
        return labels_dict             


In [3]:
test_dir = "./test/"  


test_dataset = PolyDataset(test_dir)

test_loader = DataLoader(test_dataset, 
                          batch_size=1,
                          shuffle=False)

In [4]:
def gen_points_dict_for_all_images(model, loader): 
    """
    create a dictionary of dictionaries, where main key is file name of image.
    Each elemnts is then a dictionary of two items: 'pts_net', 'pts_tgt'
    each such item is a list of points of the form: [[x1,y1], [x2,y2],....]
    
    Args:
        model     - network for which polynimials are examined
        loader    - input data to use 
    """
    
    model.eval()  # eval mode (batchnorm uses moving mean/variance instead of mini-batch mean/variance)
                  # (dropout is set to zero)
    
    points_dict = {}  
        
    k=0
    print ("generating points_dict for all images:")
    
    for data in loader:
        # get inputs
        inputs = (data['image']).to(device)
        labels = (data['labels']).to(device)
        img_fnames = data['fname'] 
      
        
        # forward
        outputs = model(inputs.float())
        curr_batch_size = np.shape(outputs)[0]
        coeffs_num = np.shape(labels)[1]  # labels shape is (batch_size, coeffs_num)
        image_size = np.shape(inputs[0])  # image_size = [3, w, h]
        _, width, height = image_size
        
        for i in range (curr_batch_size): 
            coeffs_net = [] 
            coeffs_tgt = []
            for curr_coeff in range(coeffs_num):
                coeffs_net.append(outputs[i, curr_coeff].item()) 
                coeffs_tgt.append(labels[i, curr_coeff].item())
                

            poly_net = np.poly1d(coeffs_net) # generate polynomial representation
            poly_tgt = np.poly1d(coeffs_tgt) # generate polynomial representation

    
            x_pts = np.arange(0.0, 1.0, 0.01/width)
            y_pts_net = poly_net(x_pts)  # execute net polynomial on x_pts
            y_pts_tgt = poly_tgt(x_pts)  # execute tgt polynomial on x_pts
            
            y_pts_net = 1 - y_pts_net  # b/c on screen y=0 is on top, now 0 is at bottom as we are used
            y_pts_tgt = 1 - y_pts_tgt  # b/c on screen y=0 is on top, now 0 is at bottom as we are used
    
            x_pts *= width
            y_pts_net *= height
            y_pts_tgt *= height
    
            
            pts_net = [list(a) for a in  zip(x_pts, y_pts_net)]
            pts_tgt = [list(a) for a in  zip(x_pts, y_pts_tgt)]

            input_fname = img_fnames[i] 
            k+=1
            print (str(k) + ".   " + input_fname)
            points_dict[input_fname] = {'pts_net': pts_net,
                                        'pts_tgt': pts_tgt} 
            
    return points_dict


def draw_poly(points_dict, out_dir):
    """
    Draws polynomials from given dictionary of dictionaries 
    
    Args:
        points_dict - dictionary. key is file name of frame (fname) and elemet is a dictionary with
                       keys: 'pts_net', 'pts_tgt'. Each of which has a list of points:(x,y)
        out_dir     - ouptut directory name (e.g.: 'draws/')
    """
    
       
    if not os.path.exists(out_dir):
        os.makedirs(out_dir)
    files = glob.glob(out_dir + '*.png')
    for f in files:
        os.remove(f) 


    print("Marking frames:")
    index = 1
    for fname in points_dict:
        orig_img = Image.open(fname)
        img = Image.new(orig_img.mode, orig_img.size, (255, 255, 255))
        
        fname_points_list_net = points_dict[fname]['pts_net']
        fname_points_list_tgt = points_dict[fname]['pts_tgt']

        fname_points_list_net = [(point[0], point[1]) for point in fname_points_list_net]
        fname_points_list_tgt = [(point[0], point[1]) for point in fname_points_list_tgt]
        draw = ImageDraw.Draw(img)
        draw.point(fname_points_list_net, (255, 0, 0, 255))  # net: red
        draw.point(fname_points_list_tgt, (0, 255, 0, 255))  # tgt: green
        
    
        img.save(out_dir + fname.split('/')[-1])
        
        print (index, out_dir + fname.split('/')[-1])
        index+=1
        
        
def draw_loader_polynomials(model, loader, out_dir):
    """
    This fucntion receives a model and a loader and for each polymonial image in the loader it draws
    the polynomial that it identifies. Both polynomials (the original (in green) and the network's (in red)) 
    are saved as an image in the given out_dir diretory.
    Args:
        model   - network for which polynimials are drawn
        loader  - input data to use 
        out_dir - ouptut directory name (e.g.: 'draws/')   
    """
    
    points_dict = gen_points_dict_for_all_images(model, loader)  
    draw_poly(points_dict, out_dir)
    return

In [5]:
class SeparableConv2d(nn.Module):  #In case it is used
    """
    Seperable convolution - you can try it out if you want
    """
    def __init__(self,in_channels, out_channels, kernel_size, stride=1, padding=0, dilation=1, bias=False):
        super(SeparableConv2d,self).__init__()

        self.conv1 = nn.Conv2d(in_channels,
                               in_channels,
                               kernel_size,
                               stride,
                               padding,
                               dilation,
                               groups=in_channels,
                               bias=bias)
        
        self.pointwise = nn.Conv2d(in_channels, 
                                   out_channels, 
                                   kernel_size=1,
                                   stride=1, 
                                   padding=0, 
                                   dilation=1, 
                                   groups=1, 
                                   bias=bias)

    
    def forward(self,x):
        x = self.conv1(x)
        x = self.pointwise(x)
        return x

# Your model definition is needed here!

In [6]:
class PolyNet(nn.Module):    # nn.Module is parent class
    def __init__(self):
        super(PolyNet, self).__init__()  # calls init of parent class
        self.layer1 = nn.Sequential(nn.Conv2d(in_channels=3,
                                              out_channels=3,
                                              kernel_size=3,
                                              stride=1),
                                     nn.ReLU(),
                                     nn.BatchNorm2d(3))
#         126
        self.layer2 = nn.Sequential(nn.Conv2d(in_channels=3,
                                              out_channels=9,
                                              kernel_size=3,
                                              stride=1),
                                              nn.ReLU(),
                                              nn.MaxPool2d(2),
                                              nn.BatchNorm2d(9))
#      62   
        self.layer3 = nn.Sequential(nn.Conv2d(in_channels=9,
                                              out_channels=9,
                                              kernel_size=1,
                                              stride=1),
                                              nn.ReLU(),
                                              nn.MaxPool2d(2),
                                              nn.BatchNorm2d(9))
    #30
        self.layer4 = nn.Sequential(nn.Conv2d(in_channels=9,
                                                  out_channels=4,
                                                  kernel_size=(1)),
                                                  nn.ReLU(),
                                                  nn.MaxPool2d(2),
                                                  nn.BatchNorm2d(4))
        
        self.layer5 = nn.Sequential(nn.Conv2d(in_channels=4,
                                                  out_channels=10,
                                                  kernel_size=(1)),
                                                  nn.Tanh(),
                                                  nn.BatchNorm2d(10))
    #15
        self.layer6 = nn.Sequential(nn.Conv2d(in_channels=10,
                                                  out_channels=3,
                                                  kernel_size=(4)),
                                                  nn.LeakyReLU(),
                                                  nn.MaxPool2d(2),
                                                  nn.BatchNorm2d(3))

#         self.layer3 = nn.Sequential(nn.MaxPool2d(2,2)
        self.fc1 = nn.Linear(6*6*3, 4)
        # ----------------------------------------------
        # implementation needed here
        # ----------------------------------------------

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.layer5(out)
        out = self.layer6(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc1(out)
        """
        Feed forward through network
        Args:
            x - input to the network
            
        Returns "out", which is the network's output
        """

        # ----------------------------------------------
        # implementation needed here
        # ----------------------------------------------

        return out

In [7]:
class PolyNet2(nn.Module):    # nn.Module is parent class
    def __init__(self):
        super(PolyNet2, self).__init__()  # calls init of parent class
        self.layer1 = nn.Sequential(nn.Conv2d(in_channels=3,
                                              out_channels=10,
                                              kernel_size=3,
                                              stride=1),
                                              nn.ReLU(),
                                              nn.BatchNorm2d(10))
                                                # 126

        self.layer2 = nn.Sequential(nn.Conv2d(in_channels=10,
                                              out_channels=9,
                                              kernel_size=3,
                                              stride=1),
                                              nn.ReLU(),
                                              # nn.MaxPool2d(2),
                                              nn.BatchNorm2d(9))
                                                # 124
            
        self.layer3 = nn.Sequential(nn.Conv2d(in_channels=9,
                                              out_channels=8,
                                              kernel_size=2,
                                              stride=1),
                                              nn.ReLU(),
                                              # nn.MaxPool2d(2),
                                              nn.BatchNorm2d(8))
                                                # 123
            
        self.layer4 = nn.Sequential(nn.Conv2d(in_channels=8,
                                              out_channels=8,
                                              kernel_size=2),
                                              nn.ReLU(),
                                              nn.MaxPool2d(2),
                                              nn.BatchNorm2d(8))
                                                # 61
        
        self.layer5 = nn.Sequential(nn.Conv2d(in_channels=8,
                                              out_channels=12,
                                              kernel_size=2),
                                              nn.ReLU(),
                                              nn.MaxPool2d(2),
                                              nn.BatchNorm2d(12))
                                                # 30        
    
        self.layer6 = nn.Sequential(nn.Conv2d(in_channels=12,
                                              out_channels=5,
                                              kernel_size=3),
                                              nn.LeakyReLU(),
                                              # nn.MaxPool2d(3),
                                              nn.BatchNorm2d(5))
                                                # 28        
        
        self.layer7 = nn.Sequential(nn.Conv2d(in_channels=5,
                                              out_channels=8,
                                              kernel_size=1),
                                              nn.ReLU(),
                                              nn.MaxPool2d(2),
                                              nn.BatchNorm2d(8))
                                                # 14
        
        self.layer8 = nn.Sequential(nn.Conv2d(in_channels=8,
                                              out_channels=3,
                                              kernel_size=3),
                                              nn.ReLU(),
                                              nn.MaxPool2d(2),
                                              nn.BatchNorm2d(3))
                                                # 6

        self.fc1 = nn.Linear(6*6*3, 4)
        # ----------------------------------------------
        # implementation needed here
        # ----------------------------------------------

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.layer5(out)
        out = self.layer6(out)
        out = self.layer7(out)
        out = self.layer8(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc1(out)
        """
        Feed forward through network
        Args:
            x - input to the network
            
        Returns "out", which is the network's output
        """

        # ----------------------------------------------
        # implementation needed here
        # ----------------------------------------------

        return out

In [8]:
model = torch.load("./saves/try/polyNet_RMSprop-11 43/model.dat")  # make sure you place here the path to your model

draw_loader_polynomials(model, test_loader, './test/draw/polyNet_RMSprop-11 43/')

model = torch.load("./saves/try/PolyNet2-11 35/model.dat")  # make sure you place here the path to your model

draw_loader_polynomials(model, test_loader, './test/draw/PolyNet2-11 35/')

# RMSprop resnet 101
# RMSprop polynet RMSprop-16 57

generating points_dict for all images:
1.   ./test//images/0000.png
2.   ./test//images/0001.png
3.   ./test//images/0002.png
4.   ./test//images/0003.png
5.   ./test//images/0004.png
6.   ./test//images/0005.png
7.   ./test//images/0006.png
8.   ./test//images/0007.png
9.   ./test//images/0008.png
10.   ./test//images/0009.png
11.   ./test//images/0010.png
12.   ./test//images/0011.png
13.   ./test//images/0012.png
14.   ./test//images/0013.png
15.   ./test//images/0014.png
16.   ./test//images/0015.png
17.   ./test//images/0016.png
18.   ./test//images/0017.png
19.   ./test//images/0018.png
20.   ./test//images/0019.png
21.   ./test//images/0020.png
22.   ./test//images/0021.png
23.   ./test//images/0022.png
24.   ./test//images/0023.png
25.   ./test//images/0024.png
26.   ./test//images/0025.png
27.   ./test//images/0026.png
28.   ./test//images/0027.png
29.   ./test//images/0028.png
30.   ./test//images/0029.png
31.   ./test//images/0030.png
32.   ./test//images/0031.png
33.   ./te

25.   ./test//images/0024.png
26.   ./test//images/0025.png
27.   ./test//images/0026.png
28.   ./test//images/0027.png
29.   ./test//images/0028.png
30.   ./test//images/0029.png
31.   ./test//images/0030.png
32.   ./test//images/0031.png
33.   ./test//images/0032.png
34.   ./test//images/0033.png
35.   ./test//images/0034.png
36.   ./test//images/0035.png
37.   ./test//images/0036.png
38.   ./test//images/0037.png
39.   ./test//images/0038.png
40.   ./test//images/0039.png
41.   ./test//images/0040.png
42.   ./test//images/0041.png
43.   ./test//images/0042.png
44.   ./test//images/0043.png
45.   ./test//images/0044.png
46.   ./test//images/0045.png
47.   ./test//images/0046.png
48.   ./test//images/0047.png
49.   ./test//images/0048.png
50.   ./test//images/0049.png
51.   ./test//images/0050.png
52.   ./test//images/0051.png
53.   ./test//images/0052.png
54.   ./test//images/0053.png
55.   ./test//images/0054.png
56.   ./test//images/0055.png
57.   ./test//images/0056.png
58.   ./te